In [ ]:
import pandas as pd
# df = pd.read_csv("universities_osm.csv")
# df = df.drop('geometry', axis=1)
# df.sample(10)

# df.to_csv("final.csv")


phar = pd.read_csv("pharmacy_osm.csv")
phar = phar.drop("geometry", axis=1)
phar.sample()

In [ ]:
import pandas as pd
import folium
import geopandas as gpd
import osmnx as ox
import numpy as np
from shapely.geometry import box

# -------------------------------
# 1. Load Islamabad boundary and create grid
# -------------------------------
place_name = "Islamabad, Pakistan"
boundary = ox.geocode_to_gdf(place_name)

minx, miny, maxx, maxy = boundary.total_bounds
step = 0.009

cells = [box(x, y, x + step, y + step)
         for x in np.arange(minx, maxx, step)
         for y in np.arange(miny, maxy, step)]

grid = gpd.GeoDataFrame(geometry=cells, crs="EPSG:4326")
clipped_grid = gpd.overlay(grid, boundary, how="intersection")
clipped_grid["grid_id"] = range(len(clipped_grid))

# -------------------------------
# 2. Load university data from CSV
# -------------------------------
df_uni = pd.read_csv("final.csv")

# -------------------------------
# 3. Initialize folium map
# -------------------------------
center = boundary.geometry.centroid.iloc[0]
m = folium.Map(location=[center.y, center.x], zoom_start=12, tiles="OpenStreetMap")

# -------------------------------
# 4. Add grid cells
# -------------------------------
for _, row in clipped_grid.iterrows():
    folium.GeoJson(
        row["geometry"],
        style_function=lambda x: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0
        },
        tooltip=f"Grid ID: {row['grid_id']}"
    ).add_to(m)

# -------------------------------
# 5. Add university markers with flag icon
# -------------------------------
for _, row in df_uni.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['name'],
        tooltip=row['name'],
        icon=folium.Icon(icon='flag', color='red')
    ).add_to(m)

# -------------------------------
# 6. Save the map
# -------------------------------
m.save("C:/Users/tehma/OneDrive/Desktop/Shop Prediction Model/SHOP_PROJECT/All_HTML_Mappings/islamabad_grid_with_universities.html")



In [ ]:
m